In [4]:
import nbaScraper as ns
import actNetScraper as ans
from datetime import datetime, timedelta

database_export = True
store_locally = True

# nba website, basketball referenece scraper
scraper = ns.scraper(
    browser_path = '..\\browser\\geckodriver.exe',
    database_export = database_export, 
    store_locally = store_locally,
    pymysql_conn_str =  None
)
today = scraper.meta_data['today_dt']
lastNgames = 10
day_adj = -1

### prop line/odd scraper
specified = []
# day adjustment from today (date of running script), negative = dates into the past
dayJump = 0 
# date can be a list of dates if multiple need scraping 'YYYY-MM-DD'
# default is to only pull today or today + dayJump
dates = [(datetime.today() + timedelta(days=dayJump)).strftime('%Y-%m-%d')]

prop_scraper = ans.actNetScraper(
    browser_path = '..\\browser\\geckodriver.exe',
    dates = dates,
    #leagues = leagues,
    database_export = database_export, 
    store_locally = store_locally,
    config_path = '..\\..\\..\\Notes-General\\config.txt'
)

# turn to False if issues loading new players
#prop_scraper.update_players = False

# update the league list to only ones with games today
leagues = prop_scraper.check_for_league_games(date_check = None, update_class_leagues_var = True)
print('scraping for', leagues, 'on', prop_scraper.run_date_str)

scraping for ['nba', 'nhl'] on 2024-12-31


In [ ]:
scraper.get_nba_team_playtype_data(
    base_url = 'https://www.nba.com/stats/teams/{playtype}?TypeGrouping={sideofball}&SeasonType={type}',
    play_types = [
        'isolation', 'transition', 'ball-handler', 'roll-man', 'playtype-post-up',
        'spot-up', 'hand-off', 'cut', 'off-screen','putbacks'
    ],
    sides = ['offensive', 'defensive'],
    season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
    database_table = 'statsteamplaytypes'
)

scraper.get_nba_team_shotzone_data(
    base_url = 'https://www.nba.com/stats/teams/{sideOfBall}?DistanceRange=By+Zone&LastNGames={lastNgames}&SeasonType={type}', 
    sides = {'offensive':'shooting', 'defensive':'opponent-shooting'},
    season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
    lastNgames = lastNgames,
    database_table = 'statsteamshotzones'
)

scraper.get_nba_player_playtype_data(
    base_url = 'https://www.nba.com/stats/players/{playtype}?TypeGrouping={sideofball}&SeasonType={type}', 
    play_types = [
        'isolation', 'transition', 'ball-handler', 'roll-man', 'playtype-post-up',
        'spot-up', 'hand-off', 'cut', 'off-screen','putbacks'
    ],
    sides = ['offensive'],
    season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
    database_table = 'statsplayerplaytypes'
)

scraper.get_nba_player_shotzone_data(
    base_url = 'https://www.nba.com/stats/players/shooting?DistanceRange=By+Zone&LastNGames={lastNgames}&SeasonType={type}', 
    season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
    lastNgames = lastNgames,
    database_table = 'statsplayershotzones'
)

scraper.get_nba_player_passing_data(
    base_url = 'https://www.nba.com/stats/players/passing?DateFrom={d1}&DateTo={d2}&LastNGames=0&PerMode=Totals&SeasonType={type}', 
    today_date = today,
    day_adjuster = day_adj,
    season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
    #lastNgames = 10,
    database_table = 'statsplayerpassing'
)

scraper.get_nba_player_rebounding_data(
    base_url = 'https://www.nba.com/stats/players/rebounding?DateFrom={d1}&DateTo={d2}&LastNGames=0&PerMode=Totals&SeasonType={type}', 
    today_date = today,
    day_adjuster = day_adj,
    season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
    #lastNgames = 10,
    database_table = 'statsplayerrebounding'
)

scraper.get_bref_pos_estimates(
        base_url = 'https://www.basketball-reference.com/teams/{team}/{season}.html#pbp', 
        today_date = today,
        season = 2025,
        database_table = 'brefmisc'
)

scraper finished....


In [ ]:
scraper.scrape_errors

In [2]:
prop_scraper.scrape(
    sleep_secs = 2, 
    specific_props = specified,
    leagues_override = None
)

prop_scraper.processScrapes(
    remove_dups = True
)

scraping nba ...
original rows:  (850, 20)
after dups removed:  (806, 20)
[]
missing props:  []
prop          ast  blk  pa  pr  pra  pts  ra  reb  sb  stl  threes
propId count   75   71  75  75   75   75  75   75  70   72      68
scraping nhl ...
original rows:  (2680, 20)
after dups removed:  (2221, 20)
['Justin Hryckowian' 'Ty Smith']
missing props:  ['gs']
prop          ast  ats  gs1st  gs2plus  gs3plus  gsLast  pts  sog
propId count  173  479    421      245       93     461  173  176


In [14]:
# scrape missed props
import random
wait_secs = random.randint(1,5)
leagues_with_missing_props = ['nhl']
missing_props = prop_scraper.scrape_errors[leagues_with_missing_props[0]]['missing_props']

prop_scraper_missing = ans.actNetScraper(
    browser_path = '..\\browser\\geckodriver.exe',
    dates = dates,
    leagues = leagues_with_missing_props,
    database_export = database_export, 
    store_locally = store_locally,
    config_path = '..\\..\\..\\Notes-General\\config.txt'
)

prop_scraper_missing.scrape(
    sleep_secs = wait_secs, 
    specific_props = missing_props,
    leagues_override = leagues_with_missing_props
)

prop_scraper_missing.processScrapes(
    remove_dups = True
)

scraping nhl ...
original rows:  (6, 20)
after dups removed:  (6, 20)
[]
missing props:  ['ast', 'ats', 'gs1st', 'gs2plus', 'gs3plus', 'gsLast', 'pts', 'sog']
prop          gs
propId count   6
